In [3]:
import os
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
from pandas.api.types import CategoricalDtype
import sqlite3
pd.set_option('display.max_columns', 500)


## WM 2018

In [37]:
matches_wm2018 = pd.read_csv('football_sources/fifa-world-cup-2018-RussianStandardTime.csv')

matches_wm2018 = matches_wm2018.drop(["Location","Group","Result","Round Number", "Date"], axis=1)

matches_wm2018_pre = matches_wm2018.iloc[:48]
pd.options.display.max_rows = 70
matches_wm2018_pre.head(70)

,Home Team,Away Team
0,Russia,Saudi Arabia
1,Egypt,Uruguay
2,Morocco,Iran
3,Portugal,Spain
4,France,Australia
5,Argentina,Iceland
6,Peru,Denmark
7,Croatia,Nigeria
8,Costa Rica,Serbia
9,Germany,Mexico


In [38]:
matches_wm2018_after = matches_wm2018.iloc[48:]
pd.options.display.max_rows = 700
matches_wm2018_after.head(70)

,Home Team,Away Team
48,Winner Group C,Runner-up Group D
49,Winner Group A,Runner-up Group B\t
50,Winner Group B,Runner-up Group A\t
51,Winner Group D,Runner-up Group C
52,Winner Group E,Runner-up Group F
53,Winner Group G,Runner-up Group H
54,Winner Group F,Runner-up Group E
55,Winner Group H,Runner-up Group G
56,To be announced,To be announced
57,To be announced,To be announced


In [6]:
matches_wm2018_pre["country_name_home"] = matches_wm2018_pre["Home Team"]
matches_wm2018_pre["country_name_away"] = matches_wm2018_pre["Away Team"]

matches_wm2018_pre = matches_wm2018_pre.drop(["Home Team","Away Team"], axis=1)

matches_2018_pre_names = matches_wm2018_pre

matches_wm2018_pre.tail(81)

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,country_name_home,country_name_away
0,Russia,Saudi Arabia
1,Egypt,Uruguay
2,Morocco,Iran
3,Portugal,Spain
4,France,Australia
5,Argentina,Iceland
6,Peru,Denmark
7,Croatia,Nigeria
8,Costa Rica,Serbia
9,Germany,Mexico


In [7]:
prepared_matches = pd.read_csv("prepared_matches.csv", low_memory = False)
prepared_matches = prepared_matches.drop(["Unnamed: 0"],  axis=1)

In [8]:
prepared_matches_train = pd.read_csv("prepared_matches_train.csv", low_memory = False)
prepared_matches_train = prepared_matches_train.drop(["Unnamed: 0"],  axis=1)

In [9]:
#List = prepared_matches["country_name_home"].values.tolist()

#countries = pd.DataFrame(prepared_matches.loc[List].country_name_home)
#countries["country_name_home"] =  pd.DataFrame(List)
#countries.shape

#for index, row in prepared_matches.iterrows():
#    countries["country_name_home"] = prepared_matches.iloc[index].country_name_home
#    countries["country_id"] =  prepared_matches.iloc[index].id_int_home

col_list = ["country_name_home", "id_int_home"]

countries = prepared_matches[col_list]
countries.shape

(15745, 2)

In [10]:
"Korea Republic" in countries["country_name_home"].tolist()

False

In [11]:
countries.loc[-1] = ["Korea Republic", 64]  # adding a row
countries.index = countries.index + 1  # shifting index
countries = countries.sort_index()  # sorting by index

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
countries = countries.drop_duplicates(keep="last")
countries.head(100)

,country_name_home,id_int_home
0,Korea Republic,64
1378,German Democratic Republic,17201
1562,England B,52786
1780,Soviet Union,17203
2088,Czechoslovakia,17202
3344,Estonia U17,26032
3804,Slovakia U21,24965
4868,Yugoslavia,17335
4876,Morocco U23,34903
5373,Micronesia,19759


In [13]:
countries.to_csv("country-ids.csv", encoding='utf-8')

In [14]:
(countries[matches_wm2018_pre.iloc[2].country_name_home == countries.country_name_home].id_int_home).values[0]

3575

In [15]:
def prepareMatches ( matches_dataframe ):
    
    matches_dataframe["id_int_home"] = 0
    matches_dataframe["id_int_away"] = 0
    
    for index, row in matches_dataframe.iterrows():
        matches_dataframe["id_int_home"][index] = (countries[matches_dataframe.iloc[index].country_name_home == countries.country_name_home].id_int_home).values[0]
        matches_dataframe["id_int_away"][index] = (countries[matches_dataframe.iloc[index].country_name_away == countries.country_name_home].id_int_home).values[0]    

    matches_dataframe = matches_dataframe.drop(["country_name_home","country_name_away"], axis=1)
    
    matches_dataframe_withInfo = pd.DataFrame()
    counter = 0;

    for index, row in matches_dataframe.iterrows():
        match_home = splitInfoHome(prepared_matches_train.loc[prepared_matches_train['id_int_home'] == matches_dataframe["id_int_home"][index]].tail(1))
        match_away = splitInfoAway(prepared_matches_train.loc[prepared_matches_train['id_int_away'] == matches_dataframe["id_int_away"][index]].tail(1))

        match_home["match_id"] = counter
        match_away["match_id"] = counter
    
        counter = counter + 1
    
        one_match = pd.merge(match_home,
                 match_away,
                 on='match_id')

        matches_dataframe_withInfo = matches_dataframe_withInfo.append(one_match)
        
    matches_dataframe_withInfo["game_days_before_wm"] = 0
    matches_dataframe_withInfo["rank_home_days_before_wm"] = 0
    matches_dataframe_withInfo["rank_away_days_before_wm"] = 0
    matches_dataframe_withInfo["home_is_winner"] = 1

    matches_dataframe_withInfo = matches_dataframe_withInfo.drop("match_id", axis=1)

    matches_dataframe_withInfo = matches_dataframe_withInfo.fillna(0)
    
    return matches_dataframe_withInfo

In [16]:
def splitInfoAway( awayLastGame ):
    awayPartOne = awayLastGame.loc[:,["id_int_away","rank_away","total_points_away","previous_points_away","rank_change_away","cur_year_avg_away","cur_year_avg_weighted_away","last_year_avg_away","last_year_avg_weighted_away","two_year_ago_avg_away","two_year_ago_weighted_away","three_year_ago_avg_away","three_year_ago_weighted_away"]]
    awayPartTwo = awayLastGame.loc[:,["ap_1_id_int_player","ap_1_height","ap_1_country_id_int","ap_2_id_int_player","ap_2_height","ap_2_country_id_int","ap_3_id_int_player","ap_3_height","ap_3_country_id_int","ap_4_id_int_player","ap_4_height","ap_4_country_id_int","ap_5_id_int_player","ap_5_height","ap_5_country_id_int","ap_6_id_int_player","ap_6_height","ap_6_country_id_int","ap_7_id_int_player","ap_7_height","ap_7_country_id_int","ap_8_id_int_player","ap_8_height","ap_8_country_id_int","ap_9_id_int_player","ap_9_height","ap_9_country_id_int","ap_10_id_int_player","ap_10_height","ap_10_country_id_int","ap_11_id_int_player","ap_11_height","ap_11_country_id_int"]]

    awayPart = awayPartOne.join(awayPartTwo)
    
    return awayPart

In [17]:
def splitInfoHome( homeLastGame ):     
    homePartOne = homeLastGame.loc[:,["id_int_home","rank_home","total_points_home","previous_points_home","rank_change_home","cur_year_avg_home","cur_year_avg_weighted_home","last_year_avg_home","last_year_avg_weighted_home","two_year_ago_avg_home","two_year_ago_weighted_home","three_year_ago_avg_home","three_year_ago_weighted_home"]]
    homePartTwo = homeLastGame.loc[:,["hp_1_id_int_player","hp_1_height","hp_1_country_id_int","hp_2_id_int_player","hp_2_height","hp_2_country_id_int","hp_3_id_int_player","hp_3_height","hp_3_country_id_int","hp_4_id_int_player","hp_4_height","hp_4_country_id_int","hp_5_id_int_player","hp_5_height","hp_5_country_id_int","hp_6_id_int_player","hp_6_height","hp_6_country_id_int","hp_7_id_int_player","hp_7_height","hp_7_country_id_int","hp_8_id_int_player","hp_8_height","hp_8_country_id_int","hp_9_id_int_player","hp_9_height","hp_9_country_id_int","hp_10_id_int_player","hp_10_height","hp_10_country_id_int","hp_11_id_int_player","hp_11_height","hp_11_country_id_int"]]

    homePart = homePartOne.join(homePartTwo)
        
    return homePart

In [18]:
def preprocessPredictions( predictions ):
    predictionsDataFrame = pd.DataFrame( index=range(len(predictions)))
    
    predictionsDataFrame["goals_home"] = 0
    predictionsDataFrame["goals_away"] = 0
    predictionsDataFrame["home_is_winner"] = 0
    
    for index, x in enumerate(predictions):
        predictions[index][0] = round(x[0])
        predictions[index][1] = round(x[1])
        predictions[index][2] = round(x[2])

        predictionsDataFrame["goals_home"][index] = x[2]
        predictionsDataFrame["goals_away"][index] = x[2] - x[1]
        predictionsDataFrame["home_is_winner"][index] = x[0]
        
    return predictionsDataFrame
    

In [19]:
matchesfromwm18 = prepareMatches( matches_wm2018_pre )
matchesfromwm18.tail(10)

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/marco/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,ap_10_country_id_int,ap_10_height,ap_10_id_int_player,ap_11_country_id_int,ap_11_height,ap_11_id_int_player,ap_1_country_id_int,ap_1_height,ap_1_id_int_player,ap_2_country_id_int,ap_2_height,ap_2_id_int_player,ap_3_country_id_int,ap_3_height,ap_3_id_int_player,ap_4_country_id_int,ap_4_height,ap_4_id_int_player,ap_5_country_id_int,ap_5_height,ap_5_id_int_player,ap_6_country_id_int,ap_6_height,ap_6_id_int_player,ap_7_country_id_int,ap_7_height,ap_7_id_int_player,ap_8_country_id_int,ap_8_height,ap_8_id_int_player,ap_9_country_id_int,ap_9_height,ap_9_id_int_player,cur_year_avg_away,cur_year_avg_home,cur_year_avg_weighted_away,cur_year_avg_weighted_home,hp_10_country_id_int,hp_10_height,hp_10_id_int_player,hp_11_country_id_int,hp_11_height,hp_11_id_int_player,hp_1_country_id_int,hp_1_height,hp_1_id_int_player,hp_2_country_id_int,hp_2_height,hp_2_id_int_player,hp_3_country_id_int,hp_3_height,hp_3_id_int_player,hp_4_country_id_int,hp_4_height,hp_4_id_int_player,hp_5_country_id_int,hp_5_height,hp_5_id_int_player,hp_6_country_id_int,hp_6_height,hp_6_id_int_player,hp_7_country_id_int,hp_7_height,hp_7_id_int_player,hp_8_country_id_int,hp_8_height,hp_8_id_int_player,hp_9_country_id_int,hp_9_height,hp_9_id_int_player,id_int_away,id_int_home,last_year_avg_away,last_year_avg_home,last_year_avg_weighted_away,last_year_avg_weighted_home,previous_points_away,previous_points_home,rank_away,rank_change_away,rank_change_home,rank_home,three_year_ago_avg_away,three_year_ago_avg_home,three_year_ago_weighted_away,three_year_ago_weighted_home,total_points_away,total_points_home,two_year_ago_avg_away,two_year_ago_avg_home,two_year_ago_weighted_away,two_year_ago_weighted_home,game_days_before_wm,rank_home_days_before_wm,rank_away_days_before_wm,home_is_winner
0,9171.0,1.68,114940.0,9171.0,1.71,50200.0,9171.0,1.92,120619.0,9171.0,1.78,298228.0,9171.0,1.75,172320.0,9171.0,1.88,81723.0,9171.0,1.90,37526.0,9171.0,1.87,54381.0,9171.0,1.73,292982.0,9171.0,1.74,203357.0,9171.0,1.73,37426.0,705.03,378.35,705.03,378.35,3433.0,1.88,213672.0,3433.0,1.81,126214.0,3433.0,1.84,128969.0,3433.0,1.70,136834.0,3433.0,1.86,147419.0,3433.0,1.92,68499.0,3433.0,1.84,145840.0,3433.0,1.78,37372.0,3433.0,1.87,28336.0,3433.0,1.79,123951.0,3433.0,1.80,13556.0,3584.0,3433.0,633.16,390.38,316.58,195.19,1103.0,662.0,10.0,2.0,7.0,43.0,208.92,87.59,41.78,17.52,1159.94,714.13,321.84,410.23,96.55,123.07,0,0,0,1
0,3375.0,1.82,296622.0,3375.0,1.86,44779.0,3375.0,1.89,59377.0,3375.0,1.70,69751.0,3375.0,1.73,138927.0,3375.0,1.94,18944.0,3375.0,1.84,25557.0,3375.0,1.74,60444.0,3375.0,1.71,7600.0,3375.0,1.77,74229.0,3375.0,1.76,85288.0,510.42,266.33,510.42,266.33,3444.0,1.83,242631.0,3444.0,1.70,168337.0,3444.0,1.96,295360.0,3444.0,1.79,178957.0,3444.0,1.87,309944.0,3444.0,1.91,192875.0,3444.0,1.89,358143.0,3444.0,1.83,274839.0,3444.0,1.73,131709.0,3444.0,1.78,45581.0,3444.0,1.77,59866.0,3437.0,3444.0,903.24,432.93,451.62,216.47,1348.0,606.0,4.0,0.0,0.0,52.0,992.63,366.71,198.53,73.34,1359.36,609.00,662.66,176.21,198.80,52.86,0,0,0,1
0,6303.0,1.77,35773.0,6303.0,1.75,50935.0,0.0,0.00,0.0,6303.0,1.83,29559.0,6303.0,1.88,64134.0,6303.0,1.84,51046.0,6303.0,1.83,52965.0,6303.0,1.70,118537.0,6303.0,1.69,20506.0,6303.0,1.75,234918.0,6303.0,1.75,316889.0,523.51,471.88,523.51,471.88,3574.0,1.88,29392.0,3574.0,1.87,115607.0,3574.0,1.86,205657.0,3574.0,1.70,42948.0,3574.0,1.90,30338.0,3574.0,1.90,187147.0,3574.0,1.88,137648.0,3574.0,1.90,130353.0,3574.0,1.83,9410.0,3574.0,1.84,34453.0,3574.0,1.86,89231.0,3556.0,3574.0,568.50,481.34,284.25,240.67,1048.0,931.0,15.0,0.0,1.0,21.0,484.16,485.96,96.83,97.19,1052.60,919.77,493.37,366.76,148.01,110.03,0,0,0,1
0,3447.0,1.76,64126.0,3447.0,1.86,130345.0,3447.0,1.98,55282.0,3447.0,1.81,148004.0,3447.0,1.83,271284.0,3447.0,1.85,95182.0,3447.0,1.87,25942.0,3447.0,1.81,48890.0,3447.0,1.74,295183.0,3447.0,1.69,148372.0,3447.0,1.67,209649.0,659.74,520.74,659.74,520.74,6303.0,1.77,35773.0,6303.0,1.75,50935.0,0.0,0.00,0.0,6303.0,1.83,29559.0,6303.0,1.88,64134.0,6303.0,

In [20]:
splitInfoAway(prepared_matches_train.loc[prepared_matches_train['id_int_away'] == 3807].tail(1))

,id_int_away,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int
15724,3807.0,69.0,493.79,522.0,-5.0,194.21,194.21,324.72,162.36,384.56,115.37,109.26,21.85,50219.0,1.93,3382.0,100986.0,1.9,3382.0,43250.0,1.89,3382.0,42710.0,1.86,3382.0,9594.0,1.93,3382.0,35047.0,1.88,3382.0,88755.0,1.81,3382.0,50202.0,1.73,3382.0,140776.0,1.8,3382.0,56416.0,1.69,3382.0,96341.0,1.91,3382.0


In [21]:
matches_2018_train = prepared_matches_train
matches_2018_train = matches_2018_train.drop("id_int", axis=1)
matches_2018_train.tail(1)


,id_int_home,id_int_away,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,class_AFC Asian Cup 2007,class_AFC Asian Cup 2011,class_AFC Asian Cup 2015,class_AFC Challenge Cup 2014,class_AFC-OFC Challenge Cup,class_Afrika-Cup 1998,class_Afrika-Cup 2000,class_Afrika-Cup 2002,class_Afrika-Cup 2004,class_Afrika-Cup 2006,class_Afrika-Cup 2008,class_Afrika-Cup 2010,class_Afrika-Cup 2012,class_Afrika-Cup 2013,class_Afrika-Cup 2015,class_Afrika-Cup 2017,class_Afrika-Cup-Qualifikation,class_Artemio Franchi Trophy,class_Asian Cup Qualifikation,class_CONCACAF Cup,class_Caribbean Cup 2005,class_Caribbean Cup 2007,class_Caribbean Cup 2008,class_Caribbean Cup 2010,class_Caribbean Cup 2012,class_Caribbean Cup 2014,class_Caribbean Cup 2017,class_Caribbean Cup Qualifikation,class_Confederations Cup 1997,class_Confederations Cup 1999,class_Confederations Cup 2001,class_Confederations Cup 2003,class_Confederations Cup 2005,class_Confederations Cup 2009,class_Confederations Cup 2013,class_Confederations Cup 2017,class_Copa América 1983,class_Copa América 1987,class_Copa América 1989,class_Copa América 1991,class_Copa América 1993,class_Copa América 1995,class_Copa América 1997,class_Copa América 1999,class_Copa América 2001,class_Copa América 2004,class_Copa América 2007,class_Copa América 2011,class_Copa América 2015,class_Copa América Centenario 2016,class_Copa América Centenario Play-In,class_Copa Centroamericana 2011,class_Copa Centroamericana 2013,class_Copa Centroamericana 2014,class_Copa Centroamericana 2017,class_EM-Qualifikation,class_EM-Qualifikation Playoffs,class_Europameisterschaft 1980,class_Europameisterschaft 1984,class_Europameisterschaft 1988,class_Europameisterschaft 1992,class_Europameisterschaft 1996,class_Europameisterschaft 2000,class_Europameisterschaft 2004,class_Europameisterschaft 2008,class_Europameisterschaft 2012,class_Europameisterschaft 2016,class_Freundschaftsspiele,class_Gold Cup 1991,class_Gold Cup 1993,class_Gold Cup 1996,class_Gold Cup 1998,class_Gold Cup 2000,class_Gold Cup 2002,class_Gold Cup 2003,class_Gold Cup 2005,class_Gold Cup 2007,class_Gold Cup 2009,class_Gold Cup 2011,class_Gold Cup 2013,class_Gold Cup 2015,class_Gold Cup 2017,class_Gold Cup Playoff,class_Gold Cup Qualifikation,class_König-Fahd-Pokal 1992,class_König-Fahd-Pokal 1995,class_OFC Nations Cup 1996,class_OFC Nations Cu

In [22]:
cols_of_interest = matches_2018_train.loc[:,"class_AFC Asian Cup 2007":"class_away_UEFA"]

matches_2018_train = matches_2018_train.drop(cols_of_interest, axis=1)
matches_2018_train.tail(1)

,id_int_home,id_int_away,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,game_days_before_wm,rank_home_days_before_wm,rank_away_days_before_wm
15744,3557.0,3436.0,1.0,0.0,23.0,889.48,889.0,0.0,507.67,507.67,414.85,207.43,378.28,113.48,304.52,60.9,12.0,1053.7,1054.0,0.0,725.7,725.7,313.01,156.5,312.44,93.73,388.85,77.77,2988935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,1.0,12.0,28.0,28.0


In [23]:
matches_2018_train_result = matches_2018_train.drop(["goals_home","goals_away"], axis=1)

matches_2018_train_goal_dif = matches_2018_train
matches_2018_train_goal_dif["goals_difference_home"] = matches_2018_train["goals_home"] - matches_2018_train["goals_away"]
matches_2018_train_goal_dif = matches_2018_train.drop(["goals_home","goals_away"], axis=1)

matches_2018_train_goals_home = matches_2018_train.drop("goals_away", axis=1)

matches_2018_train_all = matches_2018_train.drop(["goals_away"], axis=1)
matches_2018_train_all["goals_difference_home"] = matches_2018_train["goals_home"] - matches_2018_train["goals_away"]

In [24]:
matches_2018_train_result.tail(1)

,id_int_home,id_int_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,game_days_before_wm,rank_home_days_before_wm,rank_away_days_before_wm
15744,3557.0,3436.0,23.0,889.48,889.0,0.0,507.67,507.67,414.85,207.43,378.28,113.48,304.52,60.9,12.0,1053.7,1054.0,0.0,725.7,725.7,313.01,156.5,312.44,93.73,388.85,77.77,2988935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,1.0,12.0,28.0,28.0


In [25]:
matches_2018_train_goal_dif.tail(1)

,id_int_home,id_int_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,game_days_before_wm,rank_home_days_before_wm,rank_away_days_before_wm,goals_difference_home
15744,3557.0,3436.0,23.0,889.48,889.0,0.0,507.67,507.67,414.85,207.43,378.28,113.48,304.52,60.9,12.0,1053.7,1054.0,0.0,725.7,725.7,313.01,156.5,312.44,93.73,388.85,77.77,2988935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,1.0,12.0,28.0,28.0,1.0


In [26]:
matches_2018_train_goals_home.tail(1)

,id_int_home,id_int_away,goals_home,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,game_days_before_wm,rank_home_days_before_wm,rank_away_days_before_wm,goals_difference_home
15744,3557.0,3436.0,1.0,23.0,889.48,889.0,0.0,507.67,507.67,414.85,207.43,378.28,113.48,304.52,60.9,12.0,1053.7,1054.0,0.0,725.7,725.7,313.01,156.5,312.44,93.73,388.85,77.77,2988935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,1.0,12.0,28.0,28.0,1.0


In [27]:
matches_2018_train_all.tail(1)

,id_int_home,id_int_away,goals_home,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,game_days_before_wm,rank_home_days_before_wm,rank_away_days_before_wm,goals_difference_home
15744,3557.0,3436.0,1.0,23.0,889.48,889.0,0.0,507.67,507.67,414.85,207.43,378.28,113.48,304.52,60.9,12.0,1053.7,1054.0,0.0,725.7,725.7,313.01,156.5,312.44,93.73,388.85,77.77,2988935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210528.0,1.88,3557.0,140678.0,1.82,3557.0,197221.0,1.8,3557.0,76051.0,1.73,3557.0,252345.0,1.8,3557.0,63422.0,1.79,3557.0,225569.0,1.75,3557.0,1.0,12.0,28.0,28.0,1.0


In [28]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# tell the models which dataframe columns are features and which one is the important label
features_result = list(matches_2018_train_all.columns.values)
label = ['home_is_winner', "goals_difference_home", "goals_home"]

features_result.remove("home_is_winner")
features_result.remove("goals_difference_home")
features_result.remove("goals_home")

#split the data into train and test
all_X = matches_2018_train_all[features_result]
all_y = matches_2018_train_all[label]

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.02)


homeIsWinner =MultiOutputRegressor(GradientBoostingRegressor()).fit(train_X, train_y).predict(matchesfromwm18)
print(homeIsWinner)

[[ 1.09874689 -2.31675137  8.50385614]
 [ 0.81130042 -4.57071342  8.78541896]
 [ 1.29094396 -3.12024838  9.9594491 ]
 [ 0.96503617 -2.75687662  9.66195819]
 [ 1.01290236 -3.65029453  8.84610914]
 [ 1.00621315 -3.20756848  8.86137948]
 [ 0.33481528 -7.57446787  3.12544642]
 [ 0.93953678 -2.52205744  8.22706721]
 [ 0.98693786 -2.31369107  8.69345746]
 [ 0.68734939 -2.54832354  7.8909236 ]
 [ 0.81761409 -2.59635866  8.51840741]
 [ 0.88712597 -3.49936978  7.93687993]
 [ 0.80067522 -2.92279988  8.9366741 ]
 [ 0.93761492 -3.50220946  8.51129125]
 [ 0.79431081 -3.80197054  8.04324346]
 [ 1.09305331 -2.43150505  9.495034  ]
 [ 0.97305223 -2.88793934  8.70903158]
 [ 0.96468464 -2.957722    8.9366741 ]
 [ 0.9718521  -2.97578459 10.00451495]
 [ 1.06912184 -3.8586609   8.08961877]
 [ 1.16417208 -3.29100341  9.23130095]
 [ 0.98211503 -4.13645128  8.15901516]
 [ 0.81209467 -3.25389701  8.12892262]
 [ 0.96118013 -3.37892129  8.51335909]
 [ 0.99230779 -2.55309043  8.50385614]
 [ 1.00731915 -3.24955209

In [72]:
preprocessPredictions( homeIsWinner ).to_csv("results_groups.csv", encoding='utf-8')

In [39]:
matches_wm2018_after.head(100)

,Home Team,Away Team
48,Winner Group C,Runner-up Group D
49,Winner Group A,Runner-up Group B\t
50,Winner Group B,Runner-up Group A\t
51,Winner Group D,Runner-up Group C
52,Winner Group E,Runner-up Group F
53,Winner Group G,Runner-up Group H
54,Winner Group F,Runner-up Group E
55,Winner Group H,Runner-up Group G
56,To be announced,To be announced
57,To be announced,To be announced


In [40]:
matches_2018_lastsixteen_names = matches_wm2018_after.loc[48:55]
matches_2018_lastsixteen_names.head(10)


,Home Team,Away Team
48,Winner Group C,Runner-up Group D
49,Winner Group A,Runner-up Group B\t
50,Winner Group B,Runner-up Group A\t
51,Winner Group D,Runner-up Group C
52,Winner Group E,Runner-up Group F
53,Winner Group G,Runner-up Group H
54,Winner Group F,Runner-up Group E
55,Winner Group H,Runner-up Group G


In [41]:
matches_2018_lastsixteen_names["Home Team"][48] = "France"
matches_2018_lastsixteen_names["Away Team"][48] = "Argentina"

matches_2018_lastsixteen_names["Home Team"][49] = "Uruguay"
matches_2018_lastsixteen_names["Away Team"][49] = "Portugal"

matches_2018_lastsixteen_names["Home Team"][50] = "Spain"
matches_2018_lastsixteen_names["Away Team"][50] = "Russia"

matches_2018_lastsixteen_names["Home Team"][51] = "Croatia"
matches_2018_lastsixteen_names["Away Team"][51] = "Denmark"

matches_2018_lastsixteen_names["Home Team"][52] = "Brazil"
matches_2018_lastsixteen_names["Away Team"][52] = "Mexico"

matches_2018_lastsixteen_names["Home Team"][53] = "Belgium"
matches_2018_lastsixteen_names["Away Team"][53] = "Japan"

matches_2018_lastsixteen_names["Home Team"][54] = "Sweden"
matches_2018_lastsixteen_names["Away Team"][54] = "Switzerland"

matches_2018_lastsixteen_names["Home Team"][55] = "Colombia"
matches_2018_lastsixteen_names["Away Team"][55] = "England"

matches_2018_lastsixteen_names.head(10)

/home/marco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Home Team,Away Team
48,France,Argentina
49,Uruguay,Portugal
50,Spain,Russia
51,Croatia,Denmark
52,Brazil,Mexico
53,Belgium,Japan
54,Sweden,Switzerland
55,Colombia,England


In [42]:
matches_2018_lastsixteen_names["country_name_home"] = matches_2018_lastsixteen_names["Home Team"]
matches_2018_lastsixteen_names["country_name_away"] = matches_2018_lastsixteen_names["Away Team"]

matches_2018_lastsixteen_names = matches_2018_lastsixteen_names.drop(["Home Team","Away Team"], axis=1)

matches_2018_lastsixteen_names.tail(81)

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,country_name_home,country_name_away
48,France,Argentina
49,Uruguay,Portugal
50,Spain,Russia
51,Croatia,Denmark
52,Brazil,Mexico
53,Belgium,Japan
54,Sweden,Switzerland
55,Colombia,England


In [43]:
matches_2018_lastsixteen_names = matches_2018_lastsixteen_names.reset_index()
matches_2018_lastsixteen_names = matches_2018_lastsixteen_names.drop('index', axis=1)
matches_2018_lastsixteen_names.head(10)

,country_name_home,country_name_away
0,France,Argentina
1,Uruguay,Portugal
2,Spain,Russia
3,Croatia,Denmark
4,Brazil,Mexico
5,Belgium,Japan
6,Sweden,Switzerland
7,Colombia,England


In [44]:
matchesfromwm18_lastsixteen = prepareMatches( matches_2018_lastsixteen_names )

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
# tell the models which dataframe columns are features and which one is the important label
features_result = list(matches_2018_train_all.columns.values)
label = ['home_is_winner', "goals_difference_home", "goals_home"]

features_result.remove("home_is_winner")
features_result.remove("goals_difference_home")
features_result.remove("goals_home")

#split the data into train and test
all_X = matches_2018_train_all[features_result]
all_y = matches_2018_train_all[label]

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.02)


homeIsWinner_lastsixteen =MultiOutputRegressor(GradientBoostingRegressor()).fit(train_X, train_y).predict(matchesfromwm18_lastsixteen)
print(homeIsWinner_lastsixteen)

[[ 0.1296475  -6.43519055  2.52562748]
 [ 0.4540803  -6.54660765  2.31537501]
 [ 0.4540803  -6.67380161  2.5402174 ]
 [ 0.41765553 -5.44503918  2.21151298]
 [ 0.34678309 -6.12736212  2.51094908]
 [ 0.4540803  -6.48534544  2.54521479]
 [ 0.34420312 -5.63040963  1.94017412]
 [ 0.48273244 -7.07204577  1.81473507]]


In [46]:
preprocessPredictions( homeIsWinner_lastsixteen ).to_csv("results_last16.csv", encoding='utf-8')

In [2037]:
matches_2018_lasteight_names = matches_wm2018_after.loc[56:59]
matches_2018_lasteight_names.head(10)

,Home Team,Away Team
56,Denmark,Uruguay
57,Portugal,Croatia
58,Brazil,Panama
59,Germany,Japan


In [2038]:
matches_2018_lasteight_names["Home Team"][56] = "Denmark"
matches_2018_lasteight_names["Away Team"][56] = "Uruguay"

matches_2018_lasteight_names["Home Team"][57] = "Portugal"
matches_2018_lasteight_names["Away Team"][57] = "Croatia"

matches_2018_lasteight_names["Home Team"][58] = "Brazil"
matches_2018_lasteight_names["Away Team"][58] = "Panama"

matches_2018_lasteight_names["Home Team"][59] = "Germany"
matches_2018_lasteight_names["Away Team"][59] = "Japan"

matches_2018_lasteight_names["country_name_home"] = matches_2018_lasteight_names["Home Team"]
matches_2018_lasteight_names["country_name_away"] = matches_2018_lasteight_names["Away Team"]

matches_2018_lasteight_names = matches_2018_lasteight_names.drop(["Home Team","Away Team"], axis=1)

matches_2018_lasteight_names.head(10)

,country_name_home,country_name_away
56,Denmark,Uruguay
57,Portugal,Croatia
58,Brazil,Panama
59,Germany,Japan


In [ ]:
matches_2018_lasteight_names = matches_2018_lasteight_names.reset_index()
matches_2018_lasteight_names = matches_2018_lasteight_names.drop('index', axis=1)

matchesfromwm18_lasteight = prepareMatches( matches_2018_lasteight_names )

In [2047]:
# tell the models which dataframe columns are features and which one is the important label
features_result = list(matches_2018_train_all.columns.values)
label = ['home_is_winner', "goals_difference_home", "goals_home"]

features_result.remove("home_is_winner")
features_result.remove("goals_difference_home")
features_result.remove("goals_home")

#split the data into train and test
all_X = matches_2018_train_all[features_result]
all_y = matches_2018_train_all[label]

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.02)


homeIsWinner_lasteight =MultiOutputRegressor(GradientBoostingRegressor()).fit(train_X, train_y).predict(matchesfromwm18_lasteight)
print(homeIsWinner_lasteight)

[[ 0.97398125 -6.45615414  2.45666623]
 [ 0.97563022 -6.23112632  1.91684215]
 [ 0.84571161 -6.45615414  2.35491603]
 [ 1.09701543 -5.94545491  2.73364841]]


In [ ]:
preprocessPredictions( homeIsWinner_lasteight ).to_csv("results_last8.csv", encoding='utf-8')

In [52]:
matches_2018_lastfour_names = matches_wm2018_after.loc[60:61]
matches_2018_lastfour_names.head(10)

,Home Team,Away Team
60,To be announced,To be announced
61,To be announced,To be announced


In [53]:
matches_2018_lastfour_names["Home Team"][60] = "France"
matches_2018_lastfour_names["Away Team"][60] = "Belgium"

matches_2018_lastfour_names["Home Team"][61] = "Croatia"
matches_2018_lastfour_names["Away Team"][61] = "England"

matches_2018_lastfour_names["country_name_home"] = matches_2018_lastfour_names["Home Team"]
matches_2018_lastfour_names["country_name_away"] = matches_2018_lastfour_names["Away Team"]

matches_2018_lastfour_names = matches_2018_lastfour_names.drop(["Home Team","Away Team"], axis=1)

matches_2018_lastfour_names.head(10)

/home/marco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

,country_name_home,country_name_away
60,France,Belgium
61,Croatia,England


In [54]:
matches_2018_lastfour_names = matches_2018_lastfour_names.reset_index()
matches_2018_lastfour_names = matches_2018_lastfour_names.drop('index', axis=1)

matchesfromwm18_lastfour = prepareMatches( matches_2018_lastfour_names )

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
# tell the models which dataframe columns are features and which one is the important label
features_result = list(matches_2018_train_all.columns.values)
label = ['home_is_winner', "goals_difference_home", "goals_home"]

features_result.remove("home_is_winner")
features_result.remove("goals_difference_home")
features_result.remove("goals_home")

#split the data into train and test
all_X = matches_2018_train_all[features_result]
all_y = matches_2018_train_all[label]

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.02)


homeIsWinner_lastfour =MultiOutputRegressor(GradientBoostingRegressor()).fit(train_X, train_y).predict(matchesfromwm18_lastfour)
print(homeIsWinner_lastfour)

[[ 0.33092225 -8.10956939  1.99983031]
 [ 0.43351394 -7.67986348  2.38390045]]


In [56]:
preprocessPredictions( homeIsWinner_lastfour ).to_csv("results_last4.csv", encoding='utf-8')

In [57]:
matches_2018_finals_names = matches_wm2018_after.loc[62:63]
matches_2018_finals_names.head(10)

,Home Team,Away Team
62,To be announced,To be announced
63,To be announced,To be announced


In [58]:
matches_2018_finals_names["Home Team"][62] = "Belgium"
matches_2018_finals_names["Away Team"][62] = "England"

matches_2018_finals_names["Home Team"][63] = "France"
matches_2018_finals_names["Away Team"][63] = "Croatia"

matches_2018_finals_names["country_name_home"] = matches_2018_finals_names["Home Team"]
matches_2018_finals_names["country_name_away"] = matches_2018_finals_names["Away Team"]

matches_2018_finals_names = matches_2018_finals_names.drop(["Home Team","Away Team"], axis=1)

matches_2018_finals_names.head(10)

/home/marco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

,country_name_home,country_name_away
62,Belgium,England
63,France,Croatia


In [59]:
matches_2018_finals_names = matches_2018_finals_names.reset_index()
matches_2018_finals_names = matches_2018_finals_names.drop('index', axis=1)

matchesfromwm18_finals = prepareMatches( matches_2018_finals_names )

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
# tell the models which dataframe columns are features and which one is the important label
features_result = list(matches_2018_train_all.columns.values)
label = ['home_is_winner', "goals_difference_home", "goals_home"]

features_result.remove("home_is_winner")
features_result.remove("goals_difference_home")
features_result.remove("goals_home")

#split the data into train and test
all_X = matches_2018_train_all[features_result]
all_y = matches_2018_train_all[label]

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.02)


homeIsWinner_finals =MultiOutputRegressor(GradientBoostingRegressor()).fit(train_X, train_y).predict(matchesfromwm18_finals)
print(homeIsWinner_finals)

In [ ]:
preprocessPredictions( homeIsWinner_finals ).to_csv("results_finals.csv", encoding='utf-8')

## Denmark will be winner